In [39]:
import pandas as pd
import requests
import datetime
import urllib.parse
from datetime import datetime
from datetime import timedelta

In [42]:
parameters={
    'from-date' : '2016-01-01',
    'to-date' : str(datetime.today())[:10],
    'q' : 'Justin%20AND%20Trudeau%20OR%20JustinTrudeau',
    'format' : 'json',
    'page' : 1,
    'page-size' : 10,
    'api-key' : "3a6cb1a7-fb92-4191-b819-fd230c164b43"
}

main_url='https://content.guardianapis.com/search?'



def extractData(params, main_urladdress):
    url=main_urladdress+ urllib.parse.urlencode(params).replace('%2520', '%20')
    data= requests.get(url).json()
    if data['response']['status']=='ok':
        return data

In [43]:
fullData = extractData(parameters,main_url)

numberofResults= fullData['response']['total']
numberofPages= fullData['response']['pages']
print('Number of results: {}'.format(numberofResults))
print('Number of pages: {}'.format(numberofPages))

Number of results: 1899
Number of pages: 190


In [44]:
def appendfullData(params, numPages, fullData):
    for currentpage in range(2, numPages+1):
        parameters['page']= currentpage
        currentData = extractData(parameters,main_url)
        if currentData['response']['results'] is not None:
            fullData['response']['results'].extend(currentData['response']['results'])
        else:
            continue

In [45]:
appendfullData(parameters, numberofPages, fullData)

In [71]:
def getTableofData(data):
    i=1
    outputDict= {}
    for results in data['response']['results']:
        if results['webPublicationDate'] is not None and results['type'] is not None and results['sectionName'] is not None:
            temp1=str(results['webPublicationDate'])
            temp2=str(results['type'])
            temp3=str(results['sectionName'])
            temp4=str(results["webUrl"])
            if i==1:
                outputDict= {'dateandType': [[temp1,temp2,temp3,temp4]]}
            else:
                outputDict['dateandType'].append([temp1,temp2,temp3,temp4])
            i=i+1
    
    labels = ['Date', 'Type', 'section','webUrl']
    dataSubset=pd.DataFrame.from_records(outputDict['dateandType'], columns=labels)
    
    return dataSubset

In [72]:
filteredData = getTableofData(fullData)

In [80]:
filteredData

,Date,Type,section,webUrl
0,2023-02-08,article,World news,https://www.theguardian.com/world/2023/feb/08/...
1,2023-03-07,article,World news,https://www.theguardian.com/world/2023/mar/07/...
2,2022-12-08,article,World news,https://www.theguardian.com/world/2022/dec/08/...
3,2023-03-24,article,World news,https://www.theguardian.com/world/2023/mar/24/...
4,2022-11-02,article,Television & radio,https://www.theguardian.com/tv-and-radio/2022/...
...,...,...,...,...
1894,2016-11-10,liveblog,US news,https://www.theguardian.com/us-news/live/2016/...
1895,2016-05-12,liveblog,Australia news,https://www.theguardian.com/australia-news/liv...
1896,2016-07-13,liveblog,Politics,https://www.theguardian.com/politics/blog/live...
1897,2016-06-17,liveblog,UK news,https://www.theguardian.com/politics/live/2016...
